In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Parkinsons_Disease"
cohort = "GSE202667"

# Input paths
in_trait_dir = "../../input/GEO/Parkinsons_Disease"
in_cohort_dir = "../../input/GEO/Parkinsons_Disease/GSE202667"

# Output paths
out_data_file = "../../output/preprocess/Parkinsons_Disease/GSE202667.csv"
out_gene_data_file = "../../output/preprocess/Parkinsons_Disease/gene_data/GSE202667.csv"
out_clinical_data_file = "../../output/preprocess/Parkinsons_Disease/clinical_data/GSE202667.csv"
json_path = "../../output/preprocess/Parkinsons_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import pandas as pd
import os
import re
import numpy as np
from typing import Optional, Dict, Any, List, Callable

# Determine data availability
is_gene_available = True  # Dataset likely contains gene expression data based on CD4+ T cells examination

# Find appropriate keys for trait, age, and gender
trait_row = 0  # "disease state" provides information about Parkinson's disease vs control
age_row = 3    # Contains age information with values like "age: 53"
gender_row = 2  # Contains gender information, though all appear to be male

# Define conversion functions for each variable
def convert_trait(value: str) -> Optional[int]:
    """Convert trait value to binary: 1 for PD, 0 for control."""
    if pd.isna(value):
        return None
    
    # Convert to string if it's not already
    value = str(value)
    
    # Extract the value after the colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if re.search(r"parkinson|pd", value.lower()):
        return 1
    elif re.search(r"healthy|control|hc", value.lower()):
        return 0
    return None

def convert_age(value: str) -> Optional[float]:
    """Convert age value to continuous numeric value."""
    if pd.isna(value):
        return None
    
    # Convert to string if it's not already
    value = str(value)
    
    # Extract the value after the colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Extract numeric value
    match = re.search(r'(\d+)', value)
    if match:
        return float(match.group(1))
    return None

def convert_gender(value: str) -> Optional[int]:
    """Convert gender value to binary: 0 for female, 1 for male."""
    if pd.isna(value):
        return None
    
    # Convert to string if it's not already
    value = str(value)
    
    # Extract the value after the colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if re.search(r"male", value.lower()):
        return 1
    elif re.search(r"female", value.lower()):
        return 0
    return None

# Validate and save cohort info for initial filtering
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Proceed with clinical feature extraction if trait data is available
if trait_row is not None:
    # Sample characteristics dictionary
    sample_chars = {
        0: ["disease state: Parkinson's disease", 'disease state: Healthy Control'], 
        1: ['cell type: CD4+ T cells'], 
        2: ['gender: male'], 
        3: ['age: 53', 'age: 57', 'age: 63', 'age: 75', 'age: 85', 'age: 76', 'age: 69', 'age: 66'], 
        4: ['Stage: 1', 'Stage: 0 (Healthy Control)', 'Stage: 4', 'Stage: 2.5'], 
        5: ['time-point post activation: 0 h', 'time-point post activation: 2 h', 'time-point post activation: 4 h', 
            'time-point post activation: 8 h', 'time-point post activation: 12 h', 'time-point post activation: 24 h']
    }
    
    # Transpose the data to create a DataFrame with samples as columns
    # This matches the expected structure for geo_select_clinical_features
    sample_ids = [f'GSM{6000000+i}' for i in range(max(len(v) for v in sample_chars.values()))]
    clinical_data = pd.DataFrame(index=sample_chars.keys(), columns=sample_ids)
    
    for row_idx, values in sample_chars.items():
        for col_idx, value in enumerate(values):
            if col_idx < len(clinical_data.columns):
                clinical_data.at[row_idx, clinical_data.columns[col_idx]] = value
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data, 
        trait=trait, 
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted data
    preview = preview_df(selected_clinical_df)
    print("Preview of Selected Clinical Features:")
    print(preview)
    
    # Save clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [ ]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


### Step 4: Gene Identifier Review

In [ ]:
# Examining the gene identifiers
# These are numeric identifiers (1, 2, 3, etc.) which are not standard human gene symbols
# Standard human gene symbols are alphanumeric like SNCA, PARK2, LRRK2, etc.
# These are likely probe IDs that need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [ ]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Observe and decide which columns to use for mapping
# The gene expression data uses numeric IDs like '1', '2', '3' as seen in the previous step
# From the gene annotation preview, we need to map the 'ID' column to 'GENE_SYMBOL' column

# 2. Extract the mapping between gene identifiers and gene symbols 
mapping_data = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='GENE_SYMBOL')

print("Gene mapping preview (first 10 rows):")
print(mapping_data.head(10))

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_data)

print(f"Shape of gene expression data after mapping: {gene_data.shape}")
print("First 10 genes after mapping:")
print(gene_data.index[:10])


### Step 7: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols from the already mapped gene expression data from Step 6
# Apply normalization to standardize gene symbols
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene expression data shape after normalization: {normalized_gene_data.shape}")
print("First 5 normalized gene symbols:")
print(normalized_gene_data.index[:5])

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load the clinical data that was already processed in Step 2
if os.path.exists(out_clinical_data_file):
    clinical_data_processed = pd.read_csv(out_clinical_data_file, index_col=0)
    print("Loaded clinical data from file")
else:
    # If the file wasn't saved, recreate the clinical features using the same parameters as in Step 2
    soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
    background_info, clinical_data = get_background_and_clinical_data(matrix_file)
    
    # Define the conversion functions from Step 2
    def convert_trait(value: str) -> Optional[int]:
        if pd.isna(value):
            return None
        value = str(value)
        if ":" in value:
            value = value.split(":", 1)[1].strip()
        if re.search(r"parkinson|pd", value.lower()):
            return 1
        elif re.search(r"healthy|control|hc", value.lower()):
            return 0
        return None

    def convert_age(value: str) -> Optional[float]:
        if pd.isna(value):
            return None
        value = str(value)
        if ":" in value:
            value = value.split(":", 1)[1].strip()
        match = re.search(r'(\d+)', value)
        if match:
            return float(match.group(1))
        return None

    def convert_gender(value: str) -> Optional[int]:
        if pd.isna(value):
            return None
        value = str(value)
        if ":" in value:
            value = value.split(":", 1)[1].strip()
        if re.search(r"male", value.lower()):
            return 1
        elif re.search(r"female", value.lower()):
            return 0
        return None
    
    clinical_data_processed = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=0,  # From Step 2
        convert_trait=convert_trait,
        age_row=3,  # From Step 2
        convert_age=convert_age,
        gender_row=2,  # From Step 2
        convert_gender=convert_gender
    )
    
    # Save it
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_data_processed.to_csv(out_clinical_data_file)

# Ensure proper structure of clinical data
if isinstance(clinical_data_processed.index, pd.RangeIndex):
    # If index is numeric range, fix the structure
    clinical_data_processed.index = [trait, 'Age', 'Gender']

print("Clinical data shape:", clinical_data_processed.shape)
print("Clinical data preview:")
print(preview_df(clinical_data_processed))

# 3. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_data_processed, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data columns (first 5):", linked_data.columns[:5])

# Print some statistics before handling missing values
print(f"Missing values per column (first 5): {linked_data.isna().sum().head()}")
print(f"Missing values in trait column: {linked_data[trait].isna().sum()} out of {len(linked_data)}")

# 4. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# 5. Determine if trait and demographic features are biased
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
print(f"Data shape after removing biased features: {linked_data.shape}")
print(f"Is the trait biased: {is_biased}")

# 6. Validate and save cohort info
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data from CD4+ T cells comparing PD patients to healthy controls."
)

# 7. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable. Linked data was not saved.")


### Step 8: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols from the already mapped gene expression data from Step 6
# Apply normalization to standardize gene symbols
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene expression data shape after normalization: {normalized_gene_data.shape}")
print("First 5 normalized gene symbols:")
print(normalized_gene_data.index[:5])

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load the clinical data that was already processed in Step 2
# We need to load the clinical data from the file that was saved in Step 2
if os.path.exists(out_clinical_data_file):
    clinical_data_processed = pd.read_csv(out_clinical_data_file, index_col=0)
    print("Loaded clinical data from file")
else:
    # If for some reason the file wasn't saved, recreate the clinical features using the same parameters as in Step 2
    soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
    background_info, clinical_data = get_background_and_clinical_data(matrix_file)
    
    # Use the same conversion function and trait_row from Step 2
    def convert_trait(value):
        """Convert occupation to binary trait (exposure risk for Parkinson's)"""
        if not isinstance(value, str):
            return None
        
        value = value.lower().split(": ")[-1].strip()
        
        if "farmworker" in value:
            return 1  # Higher pesticide exposure (risk factor for Parkinson's)
        elif "manual worker" in value:
            return 0  # Lower pesticide exposure 
        else:
            return None
    
    # Use the exact same parameters as we determined in Step 2
    clinical_data_processed = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=0,  # From Step 2
        convert_trait=convert_trait,
        age_row=None,  # From Step 2
        convert_age=None,
        gender_row=None,  # From Step 2
        convert_gender=None
    )
    
    # Save it again just to be sure
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_data_processed.to_csv(out_clinical_data_file)

print("Clinical data shape:", clinical_data_processed.shape)
print("Clinical data preview:")
print(preview_df(clinical_data_processed))

# 3. Link clinical and genetic data
# The clinical data is oriented with genes/traits as rows and samples as columns
# Transpose the normalized gene data to match this orientation (samples as columns)
genetic_data_t = normalized_gene_data

# Link clinical and genetic data vertically
linked_data = geo_link_clinical_genetic_data(clinical_data_processed, genetic_data_t)
print(f"Linked data shape: {linked_data.shape}")

# 4. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# 5. Determine if trait and demographic features are biased
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
print(f"Data shape after removing biased features: {linked_data.shape}")
print(f"Is the trait biased: {is_biased}")

# 6. Validate and save cohort info
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data from blood samples comparing farmworkers (with higher pesticide exposure, a risk factor for Parkinson's) to manual workers."
)

# 7. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable. Linked data was not saved.")